In [1]:
from dataset_loader import IEMOCAPLoader

TRANSCRIPT_DIR = "C:\\Users\\aryan\\Documents\\Study\\Research\\IEMOCAP_full_release\\Session1\\dialog\\transcriptions"
AUDIO_DIR = "C:\\Users\\aryan\\Documents\\Study\\Research\\Audio_Output"
VIDEO_DIR = "C:\\Users\\aryan\\Documents\\Study\\Research\\Video_Output"
LABEL_FILE = "C:\\Users\\aryan\\Documents\\Study\\Research\\scene_emotions.csv"

loader = IEMOCAPLoader(TRANSCRIPT_DIR, AUDIO_DIR, VIDEO_DIR, LABEL_FILE)
dataset = loader.get_dataset()

Missing audio or video for Ses01F_impro04, skipping.
Missing audio or video for Ses01F_impro05, skipping.
Missing audio or video for Ses01F_impro06, skipping.
Missing audio or video for Ses01F_impro07, skipping.
Missing audio or video for Ses01F_script01_1, skipping.
Missing audio or video for Ses01F_script01_2, skipping.
Missing audio or video for Ses01F_script01_3, skipping.
Missing audio or video for Ses01F_script02_1, skipping.
Missing audio or video for Ses01F_script02_2, skipping.
Missing audio or video for Ses01F_script03_1, skipping.
Missing audio or video for Ses01F_script03_2, skipping.
Missing audio or video for Ses01M_impro01, skipping.
Missing audio or video for Ses01M_impro02, skipping.
Missing audio or video for Ses01M_impro03, skipping.
Missing audio or video for Ses01M_impro04, skipping.
Missing audio or video for Ses01M_impro05, skipping.
Missing audio or video for Ses01M_impro06, skipping.
Missing audio or video for Ses01M_impro07, skipping.
Missing audio or video fo

In [2]:
print(f"Loaded {len(dataset)} scenes.")
print("Sample entry:", dataset[0])

Loaded 3 scenes.
Sample entry: {'scene_id': 'Ses01F_impro01', 'transcript': "Excuse me.\nDo you have your forms?\nYeah.\nLet me see them.\nIs there a problem?\nWho told you to get in this line?\nYou did.\nYou were standing at the beginning and you directed me.\nOkay. But I didn't tell you to get in this line if you are filling out this particular form.\nWell what's the problem?  Let me change it.\nThis form is a Z.X.four.\nYou can't--  This is not the line for Z.X.four.  If you're going to fill out the Z.X.four, you need to have a different form of ID.\nWhat?  I'm getting an ID.  This is why I'm here.  My wallet was stolen.\nNo. I need another set of ID to prove this is actually you.\nHow am I supposed to get an ID without an ID?  How does a person get an ID in the first place?\nI don't know.  But I need an ID to pass this form along.  I can't just send it along without an ID.\nI'm here to get an ID.\nNo.  I need another ID, a separate one.\nLike what?  Like a birth certificate?\nA bir

In [3]:
from text_embedding import BERTFeatureExtractor

In [4]:
# Extract just the transcript texts
texts = [item["transcript"] for item in dataset]

extractor = BERTFeatureExtractor()
text_embeddings = extractor.extract_features(texts)

print("Extracted features shape:", text_embeddings.shape)

Extracted features shape: torch.Size([3, 256, 768])


In [5]:
video_embedding_paths = [item["video_path"] for item in dataset]
audio_embedding_paths = [item["audio_path"] for item in dataset]
video_embeddings = []
audio_embeddings = []

In [ ]:
import os
import numpy as np
import torch

for path in video_embedding_paths:
    if os.path.exists(path):
        try:
            embedding = np.load(path)
            video_embeddings.append(embedding)
            # print(f"Loaded video embedding from: {path} (Shape: {embedding.shape})")
        except Exception as e:
            print(f"Error loading video embedding from {path}: {e}")
    else:
        print(f"Video embedding file not found: {path}")
print("\nLoaded video embeddings")

for path in audio_embedding_paths:
    if os.path.exists(path):
        try:
            embedding = np.load(path)
            audio_embeddings.append(embedding)
            # print(f"Loaded audio embedding from: {path} (Shape: {embedding.shape})")
        except Exception as e:
            print(f"Error loading audio embedding from {path}: {e}")
    else:
        print(f"Audio embedding file not found: {path}")
print("\nLoaded audio embeddings")


Loaded video embeddings

Loaded audio embeddings


In [ ]:
print(f"\nSuccessfully loaded {len(video_embeddings)} video embeddings.")
print(f"Successfully loaded {len(audio_embeddings)} audio embeddings.")
print("\nShape of the loaded video embeddings:", video_embeddings[0].shape)
print("Shape of the loaded audio embeddings:", audio_embeddings[0].shape)

print("\nShape of the loaded video embeddings:", video_embeddings[1].shape)
print("Shape of the loaded audio embeddings:", audio_embeddings[1].shape)

print("\nShape of the loaded video embeddings:", video_embeddings[2].shape)
print("Shape of the loaded audio embeddings:", audio_embeddings[2].shape)


Successfully loaded 3 video embeddings.
Successfully loaded 3 audio embeddings.

Shape of the loaded video embeddings: torch.Size([674, 768])
Shape of the loaded audio embeddings: torch.Size([139, 768])


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

In [ ]:
cls_embedding = model.embeddings.word_embeddings(torch.tensor([tokenizer.cls_token_id]))
sep_embedding = model.embeddings.word_embeddings(torch.tensor([tokenizer.sep_token_id]))

In [ ]:
print("Shape of the CLS embedding:", cls_embedding.shape)
print("Shape of the SEP embedding:", sep_embedding.shape)

Shape of the CLS embedding: torch.Size([1, 768])
Shape of the SEP embedding: torch.Size([1, 768])


In [ ]:
concatenated_embeddings = torch.cat(
    (
        cls_embedding,
        video_embeddings,
        sep_embedding,
        audio_embeddings,
        sep_embedding, # Another SEP token
        text_embeddings,
        sep_embedding # Final SEP token for the entire sequence
    ),
    dim=0 # Concatenate along the sequence dimension (axis 0)
)

TypeError: expected Tensor as element 1 in argument 0, but got list